In [1]:
import orjson

from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps
import shapely
from shapely.wkb import loads as load_wkb
import random

In [2]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [3]:
db_path = 'sqlite:////tmp/blog/spatialite_core_orm.db'
engine = create_engine(db_path,echo=True)

In [4]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
    dbapi_connection.execute('SELECT InitSpatialMetaData(1);')


In [5]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))
    centroid = Column(Geometry('POINT'))


    @property
    def shapely_geom(self):
        return load_wkb(self.geom.desc) if self.geom else None

/tmp/ipykernel_34877/3638041328.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [6]:
# Create the table
Base.metadata.create_all(engine)

2024-11-06 22:37:26,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-06 22:37:26,629 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("geometries")
2024-11-06 22:37:26,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-06 22:37:26,631 INFO sqlalchemy.engine.Engine COMMIT


InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


In [7]:
%%time
# -- orm approach
from sqlalchemy.orm import Session

# Getting the total number of rows
with Session(engine) as session:
    total_rows = session.query(GeometryModel).count()
print(total_rows)    

2024-11-06 22:37:26,754 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-06 22:37:26,759 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT geometries.id AS geometries_id, geometries.name AS geometries_name, geometries.geom AS geometries_geom, geometries.centroid AS geometries_centroid 
FROM geometries) AS anon_1
2024-11-06 22:37:26,759 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()
2024-11-06 22:37:27,308 INFO sqlalchemy.engine.Engine ROLLBACK
1063260
CPU times: user 73.6 ms, sys: 484 ms, total: 558 ms
Wall time: 556 ms


#### Query Spatial Intersection 

## Core

### Geom

Query Spatial Count-only Intersection 

In [ ]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT count(geom) 
        FROM geometries 
        WHERE geometries.ROWID IN (
        SELECT ROWID
        FROM SpatialIndex
        WHERE f_table_name = 'geometries'
            AND f_geometry_column = 'geom'
           AND search_frame =             BuildMBR(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size}
            ))
        ''')
     ).fetchone()


#print(npolys, end= " " )

id only

In [ ]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT id
        FROM geometries 
        WHERE geometries.ROWID IN (
        SELECT ROWID
        FROM SpatialIndex
        WHERE f_table_name = 'geometries'
            AND f_geometry_column = 'geom'
           AND search_frame =             BuildMBR(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size}
            ))
        ''')
     ).fetchall()


#print(len(npolys), end= " " )

return with geojson

In [ ]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

 
    npolys = conn.execute(
        text(f'''
        SELECT AsGeoJSON(geom)
        FROM geometries 
        WHERE geometries.ROWID IN (
        SELECT ROWID
        FROM SpatialIndex
        WHERE f_table_name = 'geometries'
            AND f_geometry_column = 'geom'
           AND search_frame =             BuildMBR(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size}
            ))
        ''')
     ).fetchall()


#print(npolys, end= " " )

### Centroid

Query Spatial Count-only Intersection 

In [ ]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT count(geom) 
        FROM geometries 
         WHERE geometries.ROWID IN (
        SELECT ROWID
        FROM SpatialIndex
        WHERE f_table_name = 'geometries'
            AND f_geometry_column = 'centroid'
           AND search_frame =             BuildMBR(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size}
            ))
        ''')
     ).fetchone()



#print(npolys, end= " " )

In [ ]:
s="(110316,) (89460,) (102888,) (97608,) (99024,) (117720,) (84444,) (128844,)"

In [ ]:
import numpy as np
s=s.replace("(","").replace(")","")
numbers = [int(num.strip()) for num in s.split(',') if num.strip()]

print(np.mean(numbers))

id only

In [ ]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT id
        FROM geometries 
        WHERE geometries.ROWID IN (
        SELECT ROWID
        FROM SpatialIndex
        WHERE f_table_name = 'geometries'
            AND f_geometry_column = 'centroid'
           AND search_frame =             BuildMBR(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size}
            ))
        ''')
     ).fetchall()



#print(npolys, end= " " )

return with geojson

In [ ]:
%%timeit
#--- Core approach
## --- this is for counts only

random_index = random.randint(1, total_rows)
    
half_bbox_size= 6000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    npolys = conn.execute(
        text(f'''
        SELECT AsGeoJSON(geom)
        FROM geometries 
               WHERE geometries.ROWID IN (
        SELECT ROWID
        FROM SpatialIndex
        WHERE f_table_name = 'geometries'
            AND f_geometry_column = 'centroid'
           AND search_frame =             BuildMBR(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size}
            ))
        ''')
     ).fetchall()



#print(npolys, end= " " )